## Video Analysis with Python

[Video Data with Python](https://www.apmonitor.com/dde/index.php/Main/VideoData) in the [Data-Driven Engineering](http://apmonitor.com/dde) online course.

<img align=left width=500px src='https://apmonitor.com/dde/uploads/Main/python_video.png'>

### Install OpenCV

Install OpenCV once and restart the kernel to use the package. Uncomment this cell to install OpenCV. It can be removed if installation is successful.

```python
pip install opencv-python
```

Packages are installed once, not every time the program runs.

In [ ]:
# pip install opencv-python

### Import Libraries for Video

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

### Download Video File

In [ ]:
import urllib.request
f = 'runner.mp4'
url = 'http://apmonitor.com/dde/uploads/Main/'+f
urllib.request.urlretrieve(url,f)

### View Video

In [ ]:
from IPython.display import Video
Video(f,width=550)

### Import Video File

In [ ]:
# Import the .mp4 video
v = cv2.VideoCapture(f)
w = int(v.get(cv2.CAP_PROP_FRAME_WIDTH))    
h = int(v.get(cv2.CAP_PROP_FRAME_HEIGHT))
v.release()
print('Dimensions:',w,h)

### Read Frames

In [ ]:
img = []
v = cv2.VideoCapture(f)
while v.isOpened():
    success, image = v.read()
    if success:
        img.append(image)
    else:
        break
v.release()
print('Frames Read:',len(img))

### Convert BGR (OpenCV) to RGB Format

In [ ]:
for i,im in enumerate(img):
    img[i] = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)

### Diplay First 9 Frames

In [ ]:
plt.figure(figsize=(10,5))
for i in range(9):
    plt.subplot(3,3,i+1); plt.imshow(img[i])
plt.tight_layout()

### Get Pose with Deep Learning

In [ ]:
# pip install mediapipe

In [ ]:
import pandas as pd
import mediapipe as mp
mpds = mp.solutions.drawing_styles
mpdu = mp.solutions.drawing_utils
mpp  = mp.solutions.pose

In [ ]:
# store results in dataframe
x = {'frame':[],\
     'Lshldr_x':[],'Lshldr_y':[],'Lshldr_z':[],'Lshldr_v':[],\
     'Lhip_x':[],'Lhip_y':[],'Lhip_z':[],'Lhip_v':[],\
     'Lknee_x':[],'Lknee_y':[],'Lknee_z':[],'Lknee_v':[]}
s = pd.DataFrame(x)

with mpp.Pose(
        min_detection_confidence=0.2,
        static_image_mode=False,
        model_complexity=2,
        smooth_landmarks=True,
        enable_segmentation=False,
        smooth_segmentation=False,
        min_tracking_confidence=0.2) as pose:
    for i,im in enumerate(img):
        im2 = cv2.cvtColor(im, cv2.COLOR_RGB2BGR)
        results = pose.process(im2)
        if not results.pose_landmarks:
            continue
        # draw landmarks on frame
        mpdu.draw_landmarks(
            im2,results.pose_landmarks,
            mpp.POSE_CONNECTIONS,
            landmark_drawing_spec=mpds.get_default_pose_landmarks_style())
        img[i] = im2
        # store values in dataframe
        row = [i]
        for j,lm in enumerate(results.pose_landmarks.landmark):            
            if j in [11,23,25]:
                row.extend([lm.x,lm.y,lm.z,lm.visibility])
        s.loc[i] = row
s.to_csv(f+'.csv')
plt.imshow(img[0][:,:,[2,1,0]])
plt.tight_layout()

### Modify Frames: Add Text

In [ ]:
font = cv2.FONT_HERSHEY_SIMPLEX
for i,im in enumerate(img):
    tm = i/30.0
    dm,ds = divmod(tm, 60)
    str_time = '{0:02d}:{1:05.2f}'.format(int(dm), ds)
    # large text at the top with timer
    black = (0,0,0); white = (255,255,255)
    cv2.putText(im,str_time,(950,170), \
            font, 6.8,black,20,cv2.LINE_AA)
    cv2.putText(im,str_time,(950,170), \
            font, 6.8,white,10,cv2.LINE_AA)
plt.imshow(img[6][:,:,[2,1,0]])
plt.tight_layout()

### Modify Frames: Resize

In [ ]:
scale = 0.5
for i,im in enumerate(img):
    img[i] = cv2.resize(im,None,fx=scale,fy=scale)

### Get New Frame Size

In [ ]:
h,w,c = img[0].shape
print('New Dimensions:',h,w)

### Display Resized Frames

In [ ]:
plt.figure(figsize=(10,7))
for i in range(9):
    plt.subplot(3,3,i+1); plt.imshow(img[i][:,:,[2,1,0]])
plt.tight_layout()

### Export Video File

In [ ]:
fnew = f[:-4]+'.webm'
# common formats:
#   avi with XVID (fast writing, larger file)
#   mp4 with MP4V or H264 (not compatible with some browsers)
#   webm with vp80 (slow writing, smaller file)
out = cv2.VideoWriter(fnew,\
                      cv2.VideoWriter_fourcc(*'vp80'),5,(w,h))
for im in img:
    out.write(im)
out.release()

### Display Modified Video

In [ ]:
Video(fnew,width=550)

### ✅ Knowledge Check

Calculate distance from the camera and an estimate of the velocity that the runner is moving away. Use the distance between the left shoulder and left hip as a scale to determine the distance. Use how the scale changes with time to determine runner velocity. Display the distance and velocity on the video frames, similar to the timer numbers. 